In [2]:
!pip install transformers
!pip install datasets
!pip install accelerate -U
!pip install -q hazm
!pip install -q clean-text[gpl]

import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification
from datasets import Dataset, DatasetDict
from collections import defaultdict, Counter

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import gc
import json
import copy
import collections

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-

In [3]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive"

Mounted at /content/drive
 AzOs9.zip			    passwords.zip
 BN_Sampling.ipynb		    Schedule.gsheet
'Colab Notebooks'		    untitlednew۲.pptx
 digimag.zip			   'صفحه‌گسترده بدون عنوان (1).gsheet'
 document_5848446482948034850.mp4  'صفحه‌گسترده بدون عنوان.gsheet'
'Mosh Code files'		   'کپی از 7-Day Weekly Planner (1-hour intervals).gsheet'


In [4]:
!unzip /content/drive/MyDrive/digimag.zip -d /content/

Archive:  /content/drive/MyDrive/digimag.zip
   creating: /content/digimag/
  inflating: /content/digimag/dev.csv  
  inflating: /content/digimag/train.csv  
  inflating: /content/digimag/test.csv  


In [91]:
train_data = pd.read_csv('/content/digimag/train.csv', encoding='utf-8', sep='\t')
dev_data = pd.read_csv('/content/digimag/dev.csv', encoding='utf-8', sep='\t')
test_data = pd.read_csv('/content/digimag/test.csv', encoding='utf-8', sep='\t')

train_data.head()

,Unnamed: 0,content,label,label_id
0,0,نمایش تبلیغ در لاک‌اسکرین تعدادی از گوشی‌های ه...,علم و تکنولوژی,3
1,1,شکست Justice League در باکس آفیس پس از بازخورد...,هنر و سینما,5
2,2,کلاسیک بینی؛ همه چیز در یک شب اتفاق افتاد فیلم...,هنر و سینما,5
3,3,اپل دوباره سراغ رنده رفته چراکه آپگرید کردن سط...,علم و تکنولوژی,3
4,4,بررسی جزء به جزء بهترین بخش Ori and the Blind ...,بازی ویدیویی,0


In [92]:
# just deleting unnamed col
train_data.drop('Unnamed: 0', axis=1, inplace=True)
dev_data.drop('Unnamed: 0', axis=1, inplace=True)
test_data.drop('Unnamed: 0', axis=1, inplace=True)
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6896 entries, 0 to 6895
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   6896 non-null   object
 1   label     6896 non-null   object
 2   label_id  6896 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 161.8+ KB


In [116]:
print('data information')
print(train_data.info(), '\n')

data information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6896 entries, 0 to 6895
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   content                       6896 non-null   object
 1   label                         6896 non-null   object
 2   label_id                      6896 non-null   int64 
 3   cleaned_content               6896 non-null   object
 4   cleaned_content_len_by_words  6896 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 269.5+ KB
None 



In [117]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()

    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)

    # normalizing
    # normalizer = hazm.Normalizer()
    # text = normalizer.normalize(text)

    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)

    text = wierd_pattern.sub(r'', text)

    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)

    return text

In [118]:
minlim, maxlim = 3, 256
# cleaning comments
train_data['cleaned_content'] = train_data['content'].apply(cleaning)


# calculate the length of comments based on their words
train_data['cleaned_content_len_by_words'] = train_data['cleaned_content'].apply(lambda t: len(hazm.word_tokenize(t)))

# remove comments with the length of fewer than three words
train_data['cleaned_content_len_by_words'] = train_data['cleaned_content_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else len_t)
train_data = train_data.dropna(subset=['cleaned_content_len_by_words'])
train_data = train_data.reset_index(drop=True)

train_data.head()

,content,label,label_id,cleaned_content,cleaned_content_len_by_words
0,نمایش تبلیغ در لاک‌اسکرین تعدادی از گوشی‌های ه...,علم و تکنولوژی,3,نمایش تبلیغ در لاک‌اسکرین تعدادی از گوشی‌های ه...,238
1,شکست Justice League در باکس آفیس پس از بازخورد...,هنر و سینما,5,شکست justice league در باکس آفیس پس از بازخورد...,392
2,کلاسیک بینی؛ همه چیز در یک شب اتفاق افتاد فیلم...,هنر و سینما,5,کلاسیک بینی؛ همه چیز در یک شب اتفاق افتاد فیلم...,683
3,اپل دوباره سراغ رنده رفته چراکه آپگرید کردن سط...,علم و تکنولوژی,3,اپل دوباره سراغ رنده رفته چراکه آپگرید کردن سط...,1814
4,بررسی جزء به جزء بهترین بخش Ori and the Blind ...,بازی ویدیویی,0,بررسی جزء به جزء بهترین بخش ori and the blind ...,1930


In [119]:
## taghche link
unique_labels = list(sorted(train_data['label_id'].unique()))
print(f'We have #{len(unique_labels)}: {unique_labels}')

fig = go.Figure()

groupby_rate = train_data.groupby('label_id')['label_id'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_rate.index)),
    y=groupby_rate.tolist(),
    text=groupby_rate.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label ids within contents',
    xaxis_title_text='Labels',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

We have #7: [0, 1, 2, 3, 4, 5, 6]


In [120]:
label_0 = train_data[train_data['label_id'] == 1]
label_1 = train_data[train_data['label_id'] == 2]
label_2 = train_data[train_data['label_id'] == 3]
label_3 = train_data[train_data['label_id'] == 4]
label_4 = train_data[train_data['label_id'] == 5]
label_5 = train_data[train_data['label_id'] == 6]
label_6 = train_data[train_data['label_id'] == 0]

cutting_point = min(len(label_0), len(label_1), len(label_2), len(label_3), len(label_4), len(label_5), len(label_6))

if cutting_point <= len(label_0):
    label_0 = label_0.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(label_1):
    label_1 = label_1.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(label_2):
    label_2 = label_2.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(label_3):
    label_3 = label_3.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(label_4):
    label_4 = label_4.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(label_5):
    label_5 = label_5.sample(n=cutting_point).reset_index(drop=True)
if cutting_point <= len(label_6):
    label_6 = label_6.sample(n=cutting_point).reset_index(drop=True)


new_data = pd.concat([label_0, label_1,label_2,label_3,label_4,label_5,label_6])
new_data = new_data.sample(frac=1).reset_index(drop=True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   content                       679 non-null    object
 1   label                         679 non-null    object
 2   label_id                      679 non-null    int64 
 3   cleaned_content               679 non-null    object
 4   cleaned_content_len_by_words  679 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 26.6+ KB


In [121]:
fig = go.Figure()

groupby_label = new_data.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [NEW DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [122]:
data = train_data[['cleaned_content', 'label']]
data.columns = ['content', 'label']
data.head()

,content,label
0,نمایش تبلیغ در لاک‌اسکرین تعدادی از گوشی‌های ه...,علم و تکنولوژی
1,شکست justice league در باکس آفیس پس از بازخورد...,هنر و سینما
2,کلاسیک بینی؛ همه چیز در یک شب اتفاق افتاد فیلم...,هنر و سینما
3,اپل دوباره سراغ رنده رفته چراکه آپگرید کردن سط...,علم و تکنولوژی
4,بررسی جزء به جزء بهترین بخش ori and the blind ...,بازی ویدیویی


In [123]:
labels = list(sorted(data['label'].unique()))
new_data['label_id'] = new_data['label'].apply(lambda t: labels.index(t))

train, test = train_test_split(new_data, test_size=0.1, random_state=1, stratify=new_data['label'])
train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['label'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['content'].values.tolist(), train['label_id'].values.tolist()
x_valid, y_valid = valid['content'].values.tolist(), valid['label_id'].values.tolist()
x_test, y_test = test['content'].values.tolist(), test['label_id'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(549, 5)
(62, 5)
(68, 5)


In [124]:
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

import torch
import torch.nn as nn
import torch.nn.functional as F

In [125]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cpu
CUDA is not available.  Training on CPU ...


In [126]:
# general config
MAX_LEN = 512
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 50
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'ViraIntelligentDataMining/AriaBERT'
OUTPUT_PATH = '/content/bert-fa-base-uncased-textClassification-digiMag/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [127]:
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'بازی ویدیویی': 0, 'راهنمای خرید': 1, 'سلامت و زیبایی': 2, 'علم و تکنولوژی': 3, 'عمومی': 4, 'هنر و سینما': 5, 'کتاب و ادبیات': 6}
id2label: {0: 'بازی ویدیویی', 1: 'راهنمای خرید', 2: 'سلامت و زیبایی', 3: 'علم و تکنولوژی', 4: 'عمومی', 5: 'هنر و سینما', 6: 'کتاب و ادبیات'}


In [128]:
from transformers import RobertaTokenizer, RobertaConfig,AutoTokenizer

# Load the tokenizer for RoBERTa
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME_OR_PATH)

# Create a configuration for RoBERTa, updating the label2id and id2label mappings
config = RobertaConfig.from_pretrained(
    MODEL_NAME_OR_PATH,
    label2id=label2id,
    id2label=id2label
)

# Print the configuration as a JSON string
print(config.to_json_string())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning:

`resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.



{
  "_name_or_path": "./MyRoBERTaConfig",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u0628\u0627\u0632\u06cc \u0648\u06cc\u062f\u06cc\u0648\u06cc\u06cc",
    "1": "\u0631\u0627\u0647\u0646\u0645\u0627\u06cc \u062e\u0631\u06cc\u062f",
    "2": "\u0633\u0644\u0627\u0645\u062a \u0648 \u0632\u06cc\u0628\u0627\u06cc\u06cc",
    "3": "\u0639\u0644\u0645 \u0648 \u062a\u06a9\u0646\u0648\u0644\u0648\u0698\u06cc",
    "4": "\u0639\u0645\u0648\u0645\u06cc",
    "5": "\u0647\u0646\u0631 \u0648 \u0633\u06cc\u0646\u0645\u0627",
    "6": "\u06a9\u062a\u0627\u0628 \u0648 \u0627\u062f\u0628\u06cc\u0627\u062a"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0628\u0627\u0632\u06cc \u0648\u06cc\u062f\u06cc\u0648\u06cc\u06cc": 0,
    "\u0631\u0627\u06

In [129]:
idx = np.random.randint(0, len(train))
sample_content = train.iloc[idx]['content']
sample_label = train.iloc[idx]['label']

print(f'Sample: \n{sample_content}\n{sample_label}')

Sample: 
فیلم ترسناک ما؛ اثر جدید جردن پیل منتقدان را به تحسین واداشت فیلم ترسناک ما (Us) جدیدترین اثر جردن پیل کارگردان فیلم برو بیرون (Get Out) که جمعه هفته گذشته در فستیوال جنوب از جنوب غربی (South by Southwest) مشهور به SXSW اکران شده و قرار است دوم فروردین ۱۳۹۸ به شکل عمومی اکران شود توسط منتقدان مورد بررسی قرار گرفته و تمام نقدها بر این فیلم مثبت بوده است‌ ما دومین فیلمی است که پیل در کارنامه کاری‌اش کارگردانی می‌کند‌ برو بیرون، اولین فیلم این کارگردان که مورد تمجید عموم منتقدان قرار گرفت، در مراسم اسکار ۲۰۱۸ موفق شد در چهار رشته از جمله بهترین فیلم و بهترین کارگردانی نامزد دریافت جایزه اسکار شده و نهایتا جایزه بهترین فیلم‌نامه غیراقتباسی را به خود اختصاص دهد‌ برو بیرون در کنار موفقیت میان منتقدان و جشنواره‌ها، در حوزه فروش نیز با فروش بیش از ۲۵۰ میلیون دلار در مجموع از نظر تجاری نیز فیلم موفقی در نظر گرفته می‌شود‌ به گفته پیل در فیلم جدیدش یک جهان و یک هیولای جدید ترسناک خلق شده است‌ پیل پیشتر به این موضوع نیز اشاره کرده بود که هیولاها و داستان‌های ترسناک بهترین راه جامعه برای د

In [130]:
tokens = tokenizer.tokenize(sample_content)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  Comment: {sample_content}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

  Comment: فیلم ترسناک ما؛ اثر جدید جردن پیل منتقدان را به تحسین واداشت فیلم ترسناک ما (Us) جدیدترین اثر جردن پیل کارگردان فیلم برو بیرون (Get Out) که جمعه هفته گذشته در فستیوال جنوب از جنوب غربی (South by Southwest) مشهور به SXSW اکران شده و قرار است دوم فروردین ۱۳۹۸ به شکل عمومی اکران شود توسط منتقدان مورد بررسی قرار گرفته و تمام نقدها بر این فیلم مثبت بوده است‌ ما دومین فیلمی است که پیل در کارنامه کاری‌اش کارگردانی می‌کند‌ برو بیرون، اولین فیلم این کارگردان که مورد تمجید عموم منتقدان قرار گرفت، در مراسم اسکار ۲۰۱۸ موفق شد در چهار رشته از جمله بهترین فیلم و بهترین کارگردانی نامزد دریافت جایزه اسکار شده و نهایتا جایزه بهترین فیلم‌نامه غیراقتباسی را به خود اختصاص دهد‌ برو بیرون در کنار موفقیت میان منتقدان و جشنواره‌ها، در حوزه فروش نیز با فروش بیش از ۲۵۰ میلیون دلار در مجموع از نظر تجاری نیز فیلم موفقی در نظر گرفته می‌شود‌ به گفته پیل در فیلم جدیدش یک جهان و یک هیولای جدید ترسناک خلق شده است‌ پیل پیشتر به این موضوع نیز اشاره کرده بود که هیولاها و داستان‌های ترسناک بهترین راه جامعه برای

In [131]:
encoding = tokenizer.encode_plus(
    sample_content,
    max_length=32,
    truncation=True,
    add_special_tokens=True, # Add '[CLS]' and '[SEP]'
    return_token_type_ids=True,
    return_attention_mask=True,
    padding='max_length',
    return_tensors='pt',  # Return PyTorch tensors
)

print(f'Keys: {encoding.keys()}\n')
for k in encoding.keys():
    print(f'{k}:\n{encoding[k]}')

Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

input_ids:
tensor([[    0,  7703,  8688,   503,   685,  2042,   705, 28988, 28411,  9261,
           348,   303, 11558, 44516,   907,  8688,   503,   488,    57,    87,
            13,  4944,  2042, 28988, 28411,  5431,   907,  2230,  1935,   488,
            43,     2]])
token_type_ids:
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])


In [132]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class DigiDataset(Dataset):
    """Create a PyTorch dataset for Taaghche."""

    def __init__(self, tokenizer, content, targets=None, max_len=64):
        self.content = content
        self.targets = targets
        self.has_target = targets is not None
        self.tokenizer = tokenizer
        self.max_len = max_len
        if self.has_target:
            unique_labels = set(targets)
            self.label_map = {label: i for i, label in enumerate(unique_labels)}
        else:
            self.label_map = {}

    def __len__(self):
        return len(self.content)

    def __getitem__(self, item):
        content = str(self.content[item])

        if self.has_target:
            target = self.targets[item]
            if isinstance(target, np.ndarray):
                target = target.item()  # Convert single-element array to scalar
            target = self.label_map.get(target, target)
            # Ensure target is numeric and map it correctly
            if isinstance(target, str):
                if target in self.label_map:
                    target = self.label_map[target]
                else:
                    raise ValueError(f"Target label '{target}' not in label map")

        encoding = self.tokenizer.encode_plus(
            content,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        inputs = {
            'content': content,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)

        return inputs

def create_data_loader(content, targets, tokenizer, max_len, batch_size):
    dataset = DigiDataset(
        content=content,
        targets=targets,
        tokenizer=tokenizer,
        max_len=max_len
    )
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Assuming train, valid, and test are pandas DataFrames
# And tokenizer is defined and loaded properly
MAX_LEN = 64
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
TEST_BATCH_SIZE = 8

train_data_loader = create_data_loader(train['content'].to_numpy(), train['label'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE)
valid_data_loader = create_data_loader(valid['content'].to_numpy(), valid['label'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE)
test_data_loader = create_data_loader(test['content'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE)

sample_data = next(iter(train_data_loader))

print(sample_data.keys())
print(sample_data['content'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'][0])


dict_keys(['content', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
['عزاداران بیل؛ آیینه\u200cی تمام\u200cنمای جامعه ایران غلامحسین ساعدی، پزشکی بود که دیوانه\u200cوار عاشق دنیای نویسندگی شد. سرانجام این حرفه را برگزید و ادبیات داستانی ایران را، طی دهه\u200cهای سی و چهل مدیون خود کرد. او سال ۱۳۱۴ در آذربایجان به دنیا آمد. فعالیت\u200cهای سیاسی\u200cاش را از سال ۱۳۳۰ آغاز کرد و مسئولیت انتشار چند روزنامه را در آذربایجان برعهده گرفت. بعد از کودتای ۲۸ مرداد دستگیر شد و چند ماهی در زندان به سر برد. بعد از آزادی از زندان به دعوت انجمن قلم آمریکا روانهٔ این کشور شد و سخنرانی\u200cهای متعددی در این کشور انجام داد. پس از انقلاب ساعدی مجبور به ترک ایران شد و در فرانسه اقامت گزید و در همانجا بر اثر خون\u200cریزی داخلی چشم از جهان بست. عزادارن بیل را غلامحسین ساعدی نوشته است. دوازده بار تجدید چاپ شده و بعد از گذشت سالیان، همچنان اثری خواندنی تلقی می\u200cشود که جریان خاصی را در خود پدید آورده است؛ چنانکه شاملو در مورد آن می\u200cگوید: ما عزاداران بیل را داریم از ساعدی، که به عقیده

In [133]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [134]:
from transformers import RobertaModel, AutoConfig
class TextClassificationModel(nn.Module):
    def __init__(self, config):
        super(TextClassificationModel, self).__init__()

        self.roberta  = RobertaModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.roberta (
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [135]:
pt_model = TextClassificationModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

Some weights of RobertaModel were not initialized from the model checkpoint at ViraIntelligentDataMining/AriaBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


pt_model <class '__main__.TextClassificationModel'>


In [136]:
# # sample data output

# sample_data_content = sample_data['content']
# sample_data_input_ids = sample_data['input_ids']
# sample_data_attention_mask = sample_data['attention_mask']
# sample_data_token_type_ids = sample_data['token_type_ids']
# sample_data_targets = sample_data['targets']

# # available for using in GPU
# sample_data_input_ids = sample_data_input_ids.to(device)
# sample_data_attention_mask = sample_data_attention_mask.to(device)
# sample_data_token_type_ids = sample_data_token_type_ids.to(device)
# sample_data_targets = sample_data_targets.to(device)


# # outputs = F.softmax(
# #     pt_model(sample_data_input_ids, sample_data_attention_mask, sample_data_token_type_ids),
# #     dim=1)

# outputs = pt_model(sample_data_input_ids, sample_data_attention_mask, sample_data_token_type_ids)
# _, preds = torch.max(outputs, dim=1)

# print(outputs[:5, :])
# print(preds[:5])

In [137]:
# Evaluation function
def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']  # Add this line

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)  # Add this line

            targets = dl['targets'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids  # Add this line
            )

            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, targets)

            losses.append(loss.item())
            y_pred.extend(preds)
            y_true.extend(targets)

    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss

# Training function
def train_op(model,
             data_loader,
             loss_fn,
             optimizer,
             scheduler,
             step=0,
             print_every_step=100,
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None,
             clip=0.0):

    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']  # Add this line

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)  # Add this line

        targets = dl['targets'].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids  # Add this line
        )

        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, targets)

        losses.append(loss.item())
        loss.backward()

        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        optimizer.step()
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)
    return train_y, train_loss, step, eval_loss_min


In [138]:
import json
import copy
import collections
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
import numpy as np
from tqdm import tqdm

optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)

def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        # nonlocal eval_loss_min
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)

        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))

            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss

        return eval_loss_min

    return eval_cb

eval_cb = eval_callback(EPOCHS, EPOCHS, OUTPUT_PATH)

for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model,
        data_loader=train_data_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        step=step,
        print_every_step=EEVERY_EPOCH,  # Fixed typo here
        eval=True,
        eval_cb=eval_cb,
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader,
        clip=CLIP)

    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

    eval_y, eval_loss = eval_op(
        model=pt_model,
        data_loader=valid_data_loader,
        loss_fn=loss_fn)

    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning

Epochs... :   0%|          | 0/3 [00:04<?, ?it/s]


TypeError: dropout(): argument 'input' (position 1) must be Tensor, not str

In [ ]:
def predict(model, comments, tokenizer, max_len=128, batch_size=32):
    data_loader = create_data_loader(comments, None, tokenizer, max_len, batch_size, None)

    predictions = []
    prediction_probs = []


    model.eval()
    with torch.no_grad():
        for dl in tqdm(data_loader, position=0):
            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            predictions.extend(preds)
            prediction_probs.extend(F.softmax(outputs, dim=1))

    predictions = torch.stack(predictions).cpu().detach().numpy()
    prediction_probs = torch.stack(prediction_probs).cpu().detach().numpy()

    return predictions, prediction_probs

In [ ]:
test_contents = test['content'].to_numpy()
preds, probs = predict(pt_model, test_contents, tokenizer, max_len=128)

print(preds.shape, probs.shape)

In [ ]:
y_test, y_pred = [label_list.index(label) for label in test['label'].values], preds

print(f'F1: {f1_score(y_test, y_pred, average="weighted")}')
print()
print(classification_report(y_test, y_pred, target_names=label_list))